Imports

In [1]:
from __future__ import print_function
import argparse
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
from torch.autograd import Variable
from typing import Tuple
from torch.utils.data import Dataset,DataLoader
from sklearn.metrics import confusion_matrix, top_k_accuracy_score
import torchvision                                                       
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.transforms import ToTensor
from torch import nn, optim

Prepare datasets

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def get_data(batch_size: int) -> Tuple[DataLoader, DataLoader]:
    
    train_dataset = torchvision.datasets.FashionMNIST(root="./",train=True,transform= ToTensor())

    test_dataset = torchvision.datasets.FashionMNIST(root="./", transform = ToTensor(), train=False)    

    train_DL = DataLoader(train_dataset, batch_size=batch_size)
    test_DL = DataLoader(test_dataset, batch_size=batch_size)
    return (train_DL, test_DL)

Define Network

In [3]:
class Net(nn.Module):
    #def __init__(self, in_channels=1):
    def __init__(self, pretrained = True):
        super(Net, self).__init__()
        """ self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10) """
        
        """ self.model = models.resnet50(pretrained=True)
        self.model.conv1 = nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, 10) """
        
        model_resnet18 = models.resnet18(pretrained=pretrained)
        self.conv1 = model_resnet18.conv1
        self.bn1 = model_resnet18.bn1
        self.relu = model_resnet18.relu
        self.maxpool = model_resnet18.maxpool
        self.layer1 = model_resnet18.layer1
        self.layer2 = model_resnet18.layer2
        self.layer3 = model_resnet18.layer3
        self.layer4 = model_resnet18.layer4
        self.avgpool = model_resnet18.avgpool
        
    def forward(self, x):
        
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        
        return x
    
fmnist_model = Net()

C:\Users\tarip\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\tarip\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Run and test model

In [4]:
fmnist_model.to(device)

error = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(fmnist_model.parameters(), lr=.001)

count = 0

epochs = 5
batch_size = 200

data_set = get_data(batch_size)

loss_list = []
iteration_list = []
accuracy_list = []

# Lists for knowing classwise accuracy
predictions_list = []
labels_list = []
for epoch in range(epochs):
    for images, labels in data_set[0]:
        # Transfering images and labels to GPU if available
        images, labels = images.to(device), labels.to(device)
    
        train = Variable(images.view(200, 1, 28, 28))
        labels = Variable(labels)
        # Forward pass 
        outputs = fmnist_model(train)
        loss = error(outputs, labels)
        
        # Initializing a gradient as 0 so there is no mixing of gradient among the batches
        optimizer.zero_grad()
        
        #Propagating the error backward
        loss.backward()
        
        # Optimizing the parameters
        optimizer.step()
    
        count += 1
    
    # Testing the model
    
        if not (count % 50):    # It's same as "if count % 50 == 0"
            total = 0
            correct = 0
        
            for images, labels in data_set[1]:
                images, labels = images.to(device), labels.to(device)
                labels_list.append(labels)
            
                test = Variable(images.view(200, 1, 28, 28))
            
                outputs = fmnist_model(test)
            
                predictions = torch.max(outputs, 1)[1].to(device)
                predictions_list.append(predictions)
                correct += (predictions == labels).sum()
            
                total += len(labels)
            
            accuracy = correct * 100 / total
            loss_list.append(loss.data)
            iteration_list.append(count)
            accuracy_list.append(accuracy)
        
        if not (count % 500):
            print("Iteration: {}, Loss: {}, Accuracy: {}%".format(count, loss.data, accuracy))
            



RuntimeError: Given groups=1, weight of size [64, 3, 7, 7], expected input[200, 1, 28, 28] to have 3 channels, but got 1 channels instead